<a href="https://colab.research.google.com/github/PremGorecki/MachineLearning/blob/main/07_lasy_losowe_zadanie8_i_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" 
8 Wczytaj zbiór danych MNIST (zob. rozdział 3.) i podziel go na podzbiory uczący, walidacyjny
i testowy (w stosunku 50 000 : 10 000 : 10 000). Następnie wytrenuj różne klasyfikatory, np. losowego
lasu, Extra-Trees czy SVM. Teraz spróbuj połączyć je w jeden zespół (korzystając z klasyfikatora
głosowania większościowego lub miękkiego), który będzie osiągał lepsze wyniki wobec
zestawu walidacyjnego w stosunku do pojedynczych klasyfikatorów. Po uzyskaniu takiego
modelu wypróbuj go na zbiorze testowym. O ile lepiej sprawuje się ten zespół w porównaniu do
poszczególnych klasyfikatorów?

Rozwiązania tych ćwiczeń znajdziesz w dodatku A.
"""

In [ ]:
"""
9. Uruchom poszczególne klasyfikatory utworzone w poprzednim ćwiczeniu w celu wyliczenia
prognoz dla zbioru walidacyjnego, a następnie utwórz nowy zbiór danych zawierający te predykcje:
każdy przykład uczący ma stanowić wektor przechowujący zbiór prognoz pochodzących z pojedynczych
klasyfikatorów dla obrazu, natomiast wartością docelową niech będzie klasa obrazu

Wytrenuj klasyfikator na tym nowym zestawie danych uczących. Gratulacje! Właśnie udało
Ci się wyuczyć mikser, który wraz z klasyfikatorami tworzy zespół kontaminacji. Sprawdź teraz
wydajność tego zespołu wobec danych testowych. Dla każdego obrazu w zbiorze testowym wylicz
prognozy za pomocą wszystkich klasyfikatorów, a następnie prześlij wyniki do miksera, aby uzyskać
przewidywania zespołu. Porównaj wyniki z utworzonym wcześniej klasyfikatorem głosującym

Rozwiązania tych ćwiczeń znajdziesz w dodatku A.

"""